In [1]:
# Transformers installation
! pip install transformers datasets
! pip install seqeval
! pip install evaluate
! pip install wandb

import numpy as np

# Preprocess Data


## Load Tokenizer and Dataset

Inspec Dataset has 3 datasets: train, test, and validation

### Data Types

*   train: [*sample*]
*   *sample*: {
  document: [string],
  doc_bio_tags: [int]
  }

In [2]:
from datasets import load_dataset

inspec = load_dataset("midas/inspec")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
example = inspec["train"][0]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

## Preprocess

We need to


*   Convert words to tokens
*   Realign the tokens and labels, since a single word may correspond to multiple tokens.
*   Ignore the special tokens [CLS] and [SEP] in the loss function.
*   Only label the first token of a given word, and assign -100 to the others.
*   Convert doc_bio_tags to integers for classificatioon



In [5]:
label_list = np.unique(example["doc_bio_tags"])

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

print('Mapping doc_bio_tag to integer:\n\n',label2id)
print('\nMapping integer to doc_bio_tag:\n\n',id2label)

Mapping doc_bio_tag to integer:

 {'B': 0, 'I': 1, 'O': 2}

Mapping integer to doc_bio_tag:

 {0: 'B', 1: 'I', 2: 'O'}


In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["document"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"doc_bio_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label2id[label[word_idx]]) # Convert BIO to integers for classification
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
# This is our tokenized dataset (a data_dict which we will convert to a TF Dataset)
tokenized_inspec = inspec.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Utils for Training

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [9]:
import evaluate

seqeval = evaluate.load("seqeval")

In [10]:
labels = example[f"doc_bio_tags"]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    log_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }
    wandb.log(log_results)

    return log_results

## Experiment Tracking with W&B

In [11]:
import wandb

wandb.login()
wandb.init(project="ppp-keyword-extraction")

wandb: Currently logged in as: benray (benray2024). Use `wandb login --relogin` to force relogin


## These variables will be used to define a custom loss with weighted cross entropy

In [12]:
from collections import Counter

#counting how many beginning keywords, middle keywords, and non-keywords there are
count_0s = 0
count_1s = 0
count_2s = 0

for listt in tokenized_inspec["train"]["labels"]:
    count_dict = Counter(listt)
    count_0s += count_dict[0]
    count_1s += count_dict[1]
    count_2s += count_dict[2]

#getting weights for weighted cross_entropy
max_ = max(count_0s,count_1s,count_2s)
weights = [max_/count_0s, max_/count_1s, max_/count_2s]

## Train with multi-GPU
Reference: https://saturncloud.io/docs/examples/python/tensorflow/qs-multi-gpu-tensorflow/

In [13]:
import tensorflow as tf
import keras
import time
from transformers import TFAutoModelForTokenClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import create_optimizer

def train_multigpu(n_epochs, base_lr, batchsize):
    num_train_steps = (len(tokenized_inspec["train"]) // batchsize) * n_epochs
    num_labels=3

    # Set up for multi-GPU training
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: %d" % strategy.num_replicas_in_sync)

    with strategy.scope():
        model = TFAutoModelForTokenClassification.from_pretrained(
                  "distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id)

        # We define our own optimizer (and lr_schedule which we do not use)
        optimizer, lr_schedule = create_optimizer(
                                                  init_lr=2e-5,
                                                  num_train_steps=num_train_steps,
                                                  weight_decay_rate=0.01,
                                                  num_warmup_steps=0,
                                                 )

        # Compute custom loss (CrossEntropyLoss with weights)
        def loss_fn(y_true, y_pred):
          loss = tf.nn.weighted_cross_entropy_with_logits(
              labels=tf.one_hot(y_true, depth=num_labels),
              logits=y_pred,
              pos_weight=tf.constant(weights)
          )
          loss = tf.reduce_mean(loss)

          return loss

        # The model is ready for training
        model.compile(loss=loss_fn, optimizer=optimizer, metrics=["accuracy"])

    # Load in our data as TF Datasets (with data_collator applied)
    train_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["train"],
                                        shuffle=True,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2).cache().shuffle(1000)

    valid_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["validation"],
                                        shuffle=False,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2)

    # Set up a callback for metrics at the end of every epoch
    # metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=valid_ds)
    # callbacks = [metric_callback]

    # Run training
    start = time.time()

    model.fit(x=train_ds, validation_data=valid_ds, epochs=n_epochs, callbacks=[wandb.keras.WandbCallback()])

    end = time.time() - start
    print("model training time", end)

    # Save the trained model
    path = "model/keras_multi/"
    tf.keras.models.save_model(model, path)
    print(f"Saved model to {path}")

In [14]:
# Call the training function with specified parameters
model_params = {
    "n_epochs": 10,
    "base_lr": 2e-5,
    "batchsize": 16
}

tester_plain = train_multigpu(**model_params)

Number of devices: 1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

Epoch 1/10
 6/62 [=>............................] - ETA: 32s - loss: 0.7829 - accuracy: 0.2491

62/62 [==============================] - ETA: 0s - loss: 0.4101 - accuracy: 0.3198

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231015_212450-1gqvgi4i/files/model-best)... Done. 5.5s


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231015_212450-1gqvgi4i/files/model-best)... Done. 4.0s


62/62 [==============================] - 67s 1s/step - loss: 0.1267 - accuracy: 0.3599 - val_loss: 0.1440 - val_accuracy: 0.3575
Epoch 5/10
62/62 [==============================] - 45s 723ms/step - loss: 0.1173 - accuracy: 0.3635 - val_loss: 0.1482 - val_accuracy: 0.3620
Epoch 6/10
62/62 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.3655

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231015_212450-1gqvgi4i/files/model-best)... Done. 7.3s


62/62 [==============================] - 70s 1s/step - loss: 0.1114 - accuracy: 0.3655 - val_loss: 0.1417 - val_accuracy: 0.3594
Epoch 7/10
62/62 [==============================] - 49s 788ms/step - loss: 0.1055 - accuracy: 0.3676 - val_loss: 0.1467 - val_accuracy: 0.3626
Epoch 8/10
62/62 [==============================] - 44s 715ms/step - loss: 0.1007 - accuracy: 0.3693 - val_loss: 0.1487 - val_accuracy: 0.3614
Epoch 9/10
62/62 [==============================] - 44s 714ms/step - loss: 0.0983 - accuracy: 0.3702 - val_loss: 0.1482 - val_accuracy: 0.3611
Epoch 10/10
62/62 [==============================] - 44s 716ms/step - loss: 0.0964 - accuracy: 0.3712 - val_loss: 0.1511 - val_accuracy: 0.3616
model training time 673.9675834178925


Saved model to model/keras_multi/
